# MLflowの導入について
ローカル環境上にmlflowを導入する方法をリサーチする

## 参考
* [サーバー構築手順 (Qita)](https://qiita.com/c60evaporator/items/e1fd57a0263a19b629d1#%E3%82%B7%E3%83%8A%E3%83%AA%E3%82%AA2-mlflow-on-localhost-with-sqlite)

## 用語メモ

* Run
  * 一回の試行（実験、学習、前処理、etc）
* Experiment
  * Runを束ねるグループ
  * 格納先は、ローカルファイル or HTTPリモートサーバ or DB
* Artifact
  * Runで得られた出力や中間生成物の保管先
  * 格納先は、ローカルファイル or S3

| 名称            | 機能                                            | 指定できるサービス                                    |
| ------------- | --------------------------------------------- | -------------------------------------------- |
| バックエンド        | ParametersやMetrics等の主要データを保存                  | ファイルストレージ, DB                      |
| Artifactストレージ | Artifact（テーブルデータや画像等の非構造化データ）を保存する            | ファイルストレージ, FTPサーバ, クラウド（S3等）, etc. |
| レジストリサーバ      | モデルとバージョン情報を保持する（MLflow Model Registryの設定が必要） | ファイルストレージ, DB等                               |


In [1]:
import mlflow
import sqlite3
import os

In [4]:
# 各種設定

DB_PATH= '../server/mlruns.db'
ARTIFACT_LOCATION = '../data/'
EXPERIMENT_NAME = '01_mlflow_test'

## トラッキングサーバーの作成

In [5]:
# サーバーの作成
os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)  # 親ディレクトリなければ作成
conn = sqlite3.connect(DB_PATH)  # バックエンド用DBを作成

# トラッキングサーバの（バックエンドの）場所を指定
TRACKING_URL = f'sqlite:///{DB_PATH}'
mlflow.set_tracking_uri(TRACKING_URL)

In [8]:
# Experimentの生成
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

# experiment IDの取得
if experiment is None:
    # 当該Experiment存在しないとき、新たに作成
    experiment_id = mlflow.create_experiment(
                            name=EXPERIMENT_NAME,
                            artifact_location=ARTIFACT_LOCATION)
else:
    # 当該Experiment存在するとき、IDを取得
    experiment_id = experiment.experiment_id

## 実験管理

以下を参考：
[mlflowで取得できるログ一覧](https://qiita.com/c60evaporator/items/e0eb1a0c521d1310d95d#%E6%89%8B%E9%A0%863-%E3%82%A8%E3%82%AF%E3%82%B9%E3%83%9A%E3%83%AA%E3%83%A1%E3%83%B3%E3%83%88%E3%81%AE%E4%BD%9C%E6%88%90)


In [12]:
with mlflow.start_run(experiment_id=experiment_id) as run:
    
    # パラメータを記録
    mlflow.log_params(
        {
            'name' : 0,
            'name' : 0
            }
        )
    
    # メトリクスを記録
    mlflow.log_params(
        {
            'name' : 0,
            'name' : 0
            }
        )
    

# ターミナル上で以下のコマンドでuiを立ち上げる

cd server

mlflow ui --backend-store-uri sqlite:///mlruns.db